In [ ]:
from __future__ import print_function
import dune.create as create
from dune.fem.view import adaptiveLeafGridView
from dune.fem.plotting import plotPointData as plot
import dune.grid as grid
import dune.fem as fem

grid = create.view("adaptive", "ALUConform", grid.cartesianDomain( (0,0), (1,1), (16,16) ), dimgrid=2 )
grid.hierarchicalGrid.globalRefine(0)
spc  = create.space( "FiniteVolume", grid, dimrange=1 )

In [ ]:
from ufl import *
from dune.ufl import Space

uflSpace = Space((grid.dimGrid, grid.dimWorld), 1)
uflSpace = Space((grid.dimGrid, grid.dimWorld), 1)
v = TrialFunction(uflSpace)
phi = TestFunction(uflSpace)
x = SpatialCoordinate(uflSpace.cell())
velo = VectorConstant(uflSpace.cell(),dim=grid.dimWorld)
n = FacetNormal(uflSpace.cell())
h = CellVolume(uflSpace.cell())

h_avg = (h('+') + h('-'))/2
un = (dot(velo, n) + abs(dot(velo, n)))/2.0

a_int = -dot(grad(v[0]), velo*phi[0]) * dx
a_vel = dot(jump(v), un('+') * phi('+') - un('-') * phi('-') ) * dS  + dot(v, un * phi) * ds
equation = a_int + a_vel == 0

In [ ]:
model = create.model("integrands", grid, equation)
scheme = create.scheme("galerkin", spc, model)

uh, _ = scheme.solve()